# オシレータークロス戦略のバックテスト（ブログ掲載用）

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas_ta as ta
import codecs
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [2]:
# 株価データの獲得
def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.columns = [col[0] if isinstance(col, tuple) else col for col in stock_data.columns]
    stock_data['Returns'] = stock_data['Close'].pct_change()
    return stock_data

In [3]:
# 指標値の計算とシグナルの生成
def generate_signal(df, osc_idx, ma_fast=9, ma_slow=21,
                    macd_fast=12, macd_slow=26, macd_signal=9, efi_period=2):
    # pandas_taライブラリによる各種指標の計算
    df['MA_Fast'] = df.ta.sma(close='Close', length=ma_fast)
    df['MA_Slow'] = df.ta.sma(close='Close', length=ma_slow)
    df.ta.macd(close='Close', fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
    df['EFI'] = ta.efi(df['Close'], df['Volume'], length=efi_period)
    # シグナルの判定
    # (1) 長短移動平均の位置関係
    df['MA_buy'] = df['MA_Fast'] > df['MA_Slow']
    df['MA_sell'] = df['MA_Fast'] < df['MA_Slow']
    # (2) MACDとMACDsignalの位置関係
    macd_name = f'{macd_fast}_{macd_slow}_{macd_signal}'
    df['MACD_buy'] = df[f'MACD_{macd_name}'] > df[f'MACDs_{macd_name}']
    df['MACD_sell'] = df[f'MACD_{macd_name}'] < df[f'MACDs_{macd_name}']
    # (3) EFIとゼロの位置関係
    df['EFI_buy'] = (df['EFI'] >= 0)
    df['EFI_sell'] = (df['EFI'] < 0)
    # 特定の手法によるシグナルの生成（ここでは買いポジションのみをとる）
    df['Signal'] = 0
    df.loc[df[f'{osc_idx}'], 'Signal'] = 1
    return df

In [4]:
# シグナルと終値からポジションを生成
def generate_position(df, price_col='Close', signal_col='Signal', position_col='Position',\
                      take_profit_pct=0.10, trailing_stop_pct=0.05):
    price_ary = df[price_col].to_numpy()
    signal_ary = df[signal_col].to_numpy()
    position_ary = np.zeros(len(df))
    # 初期設定
    entry_price = None # 買い参入時の価格
    in_position = False # 買いポジションを保持した状態かどうか
    trailing_stop = None # 現在の損切レベル
    take_profit = None # 利確レベル
    entry_index = None # 買いポジションをとった時点
    # シグナルと終値からポジションを生成する
    for i in range(len(df)):
        price = price_ary[i]
        position_ary[i] = int(in_position)
        # 買いポジションを保持しておらず買い条件を充足したとき
        if not in_position and signal_ary[i] == 1 :
            entry_price = price
            take_profit = entry_price * (1 + take_profit_pct)
            trailing_stop = entry_price * (1 - trailing_stop_pct)
            in_position = True
            position_ary[i] = 1
            entry_index = i
            continue
        # 買いポジションを保持しているとき
        if in_position:
            new_trailing_stop = price * (1 - trailing_stop_pct)
            trailing_stop = np.maximum(trailing_stop, new_trailing_stop) # 損切基準が上がったら書き換える
            # 利確又は損切で手仕舞い
            if price >= take_profit or price <= trailing_stop:
                in_position = False
                position_ary[i] = 0
                entry_price = None
                continue
    df[position_col] = position_ary
    return df

In [5]:
# バックテストの実施
def backtest_strategy(df, initial_balance=10000):
    # その日のポジションの符号と株価の変化率を乗じてリターンとする
    df['Daily_Return'] = df['Returns'] * df['Position'].shift()
    df['Portfolio_Value'] = (1 + df['Daily_Return']).cumprod() * initial_balance
    df.at[df.index[0], 'Portfolio_Value'] = initial_balance
    return df

# 結果のプロット
def plot_results(df):
    fig, ax1 = plt.subplots(figsize=(14, 7))
    # Plot portfolio value on the primary y-axis
    ax1.plot(df.index, df['Portfolio_Value'], label='Portfolio Value', color='deepskyblue')
    ax1.set_ylabel('Portfolio Value', color='deepskyblue')
    ax1.tick_params(axis='y', labelcolor='deepskyblue')
    # Create a secondary y-axis for stock price
    ax2 = ax1.twinx()
    # Plotting the close price with the color corresponding to the trend
    for i in range(1, len(df)):
        ax2.plot(df.index[i-1:i+1], df['Close'].iloc[i-1:i+1], 
             color='darkgreen' if df['Position'].iloc[i] == 1 else 
                   ('red' if df['Position'].iloc[i] == -1 else 'darkgrey'), linewidth=2)
    ax2.set_ylabel('Stock Price', color='gray')
    ax2.tick_params(axis='y', labelcolor='gray')
    # Add title and legend
    fig.suptitle(ticker + ' Backtest (Green:buy Gray:nutral)')
    ax1.set_xlabel('Date')
    ax1.legend(loc='upper left')
    plt.show()

# パフォーマンスの評価
def evaluate_performance(df):
    total_return = df['Portfolio_Value'].iloc[-1] / df['Portfolio_Value'].iloc[0] - 1
    volatility = df['Daily_Return'].std() * (252**0.5)
    sharpe_ratio = (df['Daily_Return'].mean() / df['Daily_Return'].std()) * (252**0.5)
    return {
        "Total Return": total_return,
        "Volatility": volatility,
        "Sharpe Ratio": sharpe_ratio
    }

In [6]:
### メインプログラム
# バックテスト期間の設定
start_date = '2020-10-01'
end_date = '2025-04-01'
# 利確と損切りの幅の設定
profit_range = 0.08
loss_range = 0.04
# シグナル判定戦略のリスト
list_oscillator = ['MA_buy','MACD_buy','EFI_buy']
# バックテスト評価用のtickerリスト
jpx = {'1306.T':"TOPIX連動", '1321.T':'日経平均連動', '1545.T':'NASDAQ連動','1546.T':'ダウ工業株30種連動',\
       '1557.T':'SP500連動', '1678.T':'Nifty50連動'}
list_ticker = jpx.keys()
# シグナル判定戦略と銘柄の全ての組み合わせでバックテストを実施
backtest_results = []
for oscillator in list_oscillator:
    for ticker in list_ticker:
        data = fetch_stock_data(ticker, start_date, end_date)
        data = generate_signal(data, osc_idx=oscillator)
        data = generate_position(data, take_profit_pct=profit_range, trailing_stop_pct=loss_range)
        data = backtest_strategy(data)
        # plot_results(data) # 多数のバックテスト実施のためプロット停止中
        performance = evaluate_performance(data)
        performance['ticker'] = ticker
        performance['osc_idx'] = oscillator
        backtest_results.append(performance)
# 蓄積した辞書型データをデータフレームに変換する
df_results = pd.DataFrame(backtest_results)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [7]:
df_results.head()

,Total Return,Volatility,Sharpe Ratio,ticker,osc_idx
0,0.247418,0.133048,0.447943,1306.T,MA_buy
1,0.494004,0.145917,0.704529,1321.T,MA_buy
2,0.165704,0.176866,0.287659,1545.T,MA_buy
3,0.662095,0.135217,0.930172,1546.T,MA_buy
4,0.830678,0.145449,1.027208,1557.T,MA_buy


In [8]:
# 行が銘柄、列が指標の表に再構築する
df_table = df_results.pivot(index='ticker',columns='osc_idx',values='Sharpe Ratio')

In [9]:
df_table

osc_idx,EFI_buy,MACD_buy,MA_buy
ticker,,,
1306.T,0.577350,0.195971,0.447943
1321.T,0.975949,0.376293,0.704529
1545.T,0.912084,0.996333,0.287659
1546.T,1.358137,1.133177,0.930172
1557.T,1.505403,1.211504,1.027208
1678.T,0.723914,0.670928,0.988381


In [10]:
# データフレームの保存
df_table.to_parquet('osc_backtest.parquet')